# Базовый пайплайн для соревнования по определению контрафакта 

In [79]:
import re
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold,StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from bs4 import BeautifulSoup
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from catboost import CatBoostClassifier
torch.cuda.is_available()

True

In [96]:
test="ml_ozon_сounterfeit_data/ml_ozon_counterfeit_test.csv"
train="ml_ozon_сounterfeit_data/ml_ozon_counterfeit_train.csv"
df_test = pd.read_csv(test, encoding="utf-8")
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22760 entries, 0 to 22759
Data columns (total 44 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            22760 non-null  int64  
 1   brand_name                    16928 non-null  object 
 2   description                   20536 non-null  object 
 3   name_rus                      22760 non-null  object 
 4   CommercialTypeName4           22760 non-null  object 
 5   rating_1_count                1707 non-null   float64
 6   rating_2_count                1707 non-null   float64
 7   rating_3_count                1707 non-null   float64
 8   rating_4_count                1707 non-null   float64
 9   rating_5_count                1707 non-null   float64
 10  comments_published_count      1707 non-null   float64
 11  photos_published_count        1707 non-null   float64
 12  videos_published_count        1707 non-null   float64
 13  P

In [97]:
df_train = pd.read_csv(train, encoding="utf-8")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197198 entries, 0 to 197197
Data columns (total 45 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   id                            197198 non-null  int64  
 1   resolution                    197198 non-null  int64  
 2   brand_name                    116667 non-null  object 
 3   description                   171138 non-null  object 
 4   name_rus                      197198 non-null  object 
 5   CommercialTypeName4           197198 non-null  object 
 6   rating_1_count                47193 non-null   float64
 7   rating_2_count                47193 non-null   float64
 8   rating_3_count                47193 non-null   float64
 9   rating_4_count                47193 non-null   float64
 10  rating_5_count                47193 non-null   float64
 11  comments_published_count      47193 non-null   float64
 12  photos_published_count        47193 non-null

In [98]:
df_train.head(10)


,id,resolution,brand_name,description,name_rus,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,...,ExemplarReturnedCountTotal30,ExemplarReturnedCountTotal90,ExemplarReturnedValueTotal7,ExemplarReturnedValueTotal30,ExemplarReturnedValueTotal90,ItemVarietyCount,ItemAvailableCount,seller_time_alive,ItemID,SellerID
0,159385,0,ACTRUM,"Мешки пылесборники для пылесоса PHILIPS, 10 шт...","Мешки для пылесоса PHILIPS TRIATLON, синтетиче...",Пылесборник,6.0,4.0,4.0,3.0,...,11.0,50.0,730.171845,896.528847,1043.118191,1.0,1.0,1860.0,78312,1218
1,288616,0,Red Line,Защитная силиконовая крышка обьектива GoPro He...,Защитная крышка Redline на экшн-камеру GoPro (...,Крышка для объектива,NaN,NaN,NaN,NaN,...,26.0,54.0,993.043882,1137.421611,1188.608000,1.0,1.0,1757.0,141999,1374
2,108090,0,Talwar Brothers,Плоский медиатор из кости толщиной 0.6 мм<br/>...,Медиатор для гитары Acura GP-PB6,Аксессуар для музыкального инструмента,0.0,0.0,1.0,0.0,...,16.0,34.0,800.822138,1174.069505,1224.798286,1.0,1.0,1722.0,53306,1448
3,415607,0,NaN,"Игра Sonic Frontiers для PlayStation 5, русски...","Игра Sonic Frontiers для PlayStation 5, русски...",Видеоигра,NaN,NaN,NaN,NaN,...,3.0,6.0,0.000000,913.530121,982.789171,3.0,3.0,1692.0,202599,715
4,332391,0,NaN,Disney Classic Games: Aladdin and The Lion Kin...,"Игра Aladdin and Lion King (PlayStation 4, анг...",Видеоигра,1.0,0.0,0.0,0.0,...,3.0,6.0,0.000000,913.542170,982.783783,3.0,3.0,1692.0,163725,715
5,139903,0,NaN,Включает:\n\n- Набор Cross Gen для Call of Dut...,Игра Call of Duty Modern Warfare-2 Cross-Gen E...,Видеоигра,NaN,NaN,NaN,NaN,...,3.0,6.0,0.000000,913.526156,982.781713,3.0,3.0,1692.0,68645,715
6,30559,0,NaN,"Нож саблевидный для мясорубки Philips HR2724, ...",Нож саблевидный для мясорубки Philips (кв. 9 мм),Запчасти для мясорубки,0.0,1.0,1.0,0.0,...,269.0,767.0,1074.568077,1215.169272,1314.483926,3.0,3.0,1683.0,15306,2759
7,319690,0,NaN,"Гайка зажимная для мясорубки под решетку Д-53,...",Гайка зажимная для для кух. комбайна Philips H...,Запчасти для мясорубки,2.0,0.0,1.0,0.0,...,269.0,767.0,1074.576260,1215.162423,1314.500236,3.0,3.0,1683.0,157310,2759
8,79124,0,NaN,Решётка для мясорубки KitFort 2101 обрезная (м...,"Решётка для мясорубки KitFort KT-2101, 2103, 2...",Запчасти для мясорубки,1.0,0.0,0.0,0.0,...,269.0,767.0,1074.568696,1215.154426,1314.507730,3.0,3.0,1683.0,39370,2759
9,393163,0,Телетай,Сим-карта Телетай тарифный план Все для общени...,Телетай SIM-карта (Вся Россия),SIM-карта,5.0,2.0,1.0,1.0,...,1.0,2.0,0.000000,462.815880,545.648989,1.0,1.0,1607.0,192392,11969


In [99]:
df_test.tail(10)

,id,brand_name,description,name_rus,CommercialTypeName4,rating_1_count,rating_2_count,rating_3_count,rating_4_count,rating_5_count,...,ExemplarReturnedCountTotal30,ExemplarReturnedCountTotal90,ExemplarReturnedValueTotal7,ExemplarReturnedValueTotal30,ExemplarReturnedValueTotal90,ItemVarietyCount,ItemAvailableCount,seller_time_alive,ItemID,SellerID
22750,303671,Aqara,Реле предназначено для автоматизации управлени...,"Aqara Реле одноканальное с нейтралью T1, модел...",Умные выключатели и реле,NaN,NaN,NaN,NaN,NaN,...,64.0,170.0,1189.324958,1286.617530,1373.399619,5.0,5.0,706.0,149460,6289
22751,365427,realme,Смартфон Realme C25Y оснащен сканером отпечат...,"realme Смартфон с25y 4/64 ГБ, бирюзовый",Смартфон,NaN,NaN,NaN,NaN,NaN,...,3.0,5.0,NaN,1148.878989,1217.750107,28.0,28.0,1014.0,179450,1635
22752,279737,Defender,Портативная акустическая система с Bluetooth 5...,Беспроводная колонка Defender G36 bluetooth 5....,Портативная акустическая система,NaN,NaN,NaN,NaN,NaN,...,97.0,101.0,1134.393984,1243.708757,1246.311668,1.0,1.0,97.0,137622,744
22753,284432,LG,86 дюймов тв;<br/><br/>телевизор лджи 86;<br/>...,"LG Телевизор OLED83G4RLA.ARUG, Smart TV, 83""...",Телевизор LG с диагональю экрана от 43 дюймов,NaN,NaN,NaN,NaN,NaN,...,4.0,4.0,1048.159080,1082.157149,1082.149426,15.0,15.0,484.0,139985,1754
22754,119035,Panasonic,Покупка картриджей и расходных материалов PANA...,"Комплект 2 шт, Картридж лазерный Panasonic KX-...","Картридж, чернила, тонер",NaN,NaN,NaN,NaN,NaN,...,105.0,293.0,1081.362025,1248.085584,1364.463899,38.0,37.0,1031.0,58403,576
22755,423002,NaN,Уникальный силиконовый 3D чехол для Этот гадже...,Защитные наушники Huawei Free Clip,Чехол для портативной акустики,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140.0,206180,9051
22756,316386,GARLYN,NaN,Пылесос вертикальный беспроводной GARLYN M-750...,Пылесос вертикальный,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.000000,0.000000,0.000000,4.0,4.0,15.0,155770,1841
22757,245410,Dell,Dell Precision T3420 — Компактная рабочая стан...,Dell Системный блок производительная рабочая с...,Настольный компьютер,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,726.445574,726.456293,726.440303,3.0,3.0,283.0,120543,9888
22758,440579,NaN,Кабель оптический для телевизора Samsung Q900R...,Кабель оптический для телевизора Samsung Q900R...,Запчасти для телевизора,NaN,NaN,NaN,NaN,NaN,...,61.0,121.0,1092.386186,1222.758381,1301.323996,239.0,239.0,287.0,214389,128
22759,371,Dyson,Европейская вилка! Переходник не требуется.Обн...,Стайлер Dyson Airwrap Complete Long Volumise H...,"Стайлер, щипцы для волос Dyson",NaN,NaN,NaN,NaN,NaN,...,6.0,9.0,1107.220017,1289.354622,1329.724542,5.0,5.0,134.0,192,154


In [100]:
# Сохраняем на диск
name=os.path.splitext(os.path.basename(test))[0]


In [17]:
# ===== ФУНКЦИЯ ОЧИСТКИ =====
def clean_text(text):
    if pd.isna(text):
        return ""
    # Убираем HTML-теги
    text = BeautifulSoup(str(text), "lxml").get_text(separator=" ")
    # Декодируем спецсимволы типа &#34;
    text = re.sub(r"&[a-zA-Z0-9#]+;", " ", text)
    # Убираем всё кроме букв, цифр и базовой пунктуации
    text = re.sub(r"[^а-яА-ЯёЁa-zA-Z0-9.,!?;:()\-\s]", " ", text)
    # Приводим к нижнему регистру
    text = text.lower()
    # Убираем лишние пробелы
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ===== ЗАГРУЗКА ДАННЫХ =====
df = df_test

# Целевые текстовые колонки
text_cols = ["brand_name", "description", "name_rus", "CommercialTypeName4"]

# ===== МОДЕЛЬ =====
MODEL_NAME = "intfloat/multilingual-e5-small"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 256   # для 4070 Ti можно поднять до 512–1024

model = SentenceTransformer(MODEL_NAME, device=DEVICE)
model.max_seq_length = 256  # подрезаем слишком длинные тексты

# Размер эмбеддинга
embed_dim = model.get_sentence_embedding_dimension()

# Подготавливаем выходной массив: [rows, cols, dim]
embeddings = np.full((len(df), len(text_cols), embed_dim), np.nan, dtype=np.float32)

# ===== ГЕНЕРАЦИЯ ЭМБЕДДИНГОВ =====
for ci, col in enumerate(text_cols):
    texts = df[col].apply(clean_text).tolist()
    
    batch_embs = []
    with torch.inference_mode():
        for i in tqdm(range(0, len(texts), BATCH_SIZE), desc=f"Encoding {col}"):
            batch = texts[i:i+BATCH_SIZE]
            # проверяем пустые строки после очистки
            valid_idx = [j for j, t in enumerate(batch) if t.strip() != ""]
            valid_texts = [batch[j] for j in valid_idx]

            if valid_texts:
                e = model.encode(valid_texts, convert_to_numpy=True, 
                                 normalize_embeddings=True, batch_size=BATCH_SIZE)
            else:
                e = []

            # собираем батч в итоговый массив
            mat = np.full((len(batch), embed_dim), np.nan, dtype=np.float32)
            if len(valid_idx) > 0:
                mat[valid_idx] = e.astype(np.float32)
            batch_embs.append(mat)
    
    embeddings[:, ci, :] = np.vstack(batch_embs)

print("Итоговая форма эмбеддингов:", embeddings.shape)
# => (197198, 4, 384)
np.save(f"Z:/PYTHON/Studying/AI/cache/{name}_embeddings.npy", embeddings)

Encoding CommercialTypeName4: 100%|████████████████████████████████████████████████████| 89/89 [00:35<00:00,  2.54it/s]


Итоговая форма эмбеддингов: (22760, 4, 384)


In [101]:
embeddings_train = np.load("Z:/PYTHON/Studying/AI/cache/text_embeddings.npy")  
print(embeddings_train.shape)  # должно быть (197000, 4, 384)

embeddings_train_flat = embeddings_train.reshape(embeddings_train.shape[0], -1)
print(embeddings_train_flat.shape)

embeddings_test = np.load(f"Z:/PYTHON/Studying/AI/cache/{name}_embeddings.npy")  
print(embeddings_test.shape)  # должно быть (197000, 4, 384)

embeddings_test_flat = embeddings_test.reshape(embeddings_test.shape[0], -1)
print(embeddings_test_flat.shape)

(197198, 4, 384)
(197198, 1536)
(22760, 4, 384)
(22760, 1536)


In [102]:
def fix_nan_embeddings(embeddings: np.ndarray) -> np.ndarray:
    """
    Заменяет NaN внутри эмбеддингов на нули.

    Args:
        embeddings: np.ndarray формы (N, D)

    Returns:
        np.ndarray: исправленная матрица эмбеддингов
    """
    if np.isnan(embeddings).any():
        embeddings = np.nan_to_num(embeddings, nan=0.0)
    return embeddings

# пример использования
embeddings_train_flat = fix_nan_embeddings(embeddings_train_flat)
embeddings_test_flat  = fix_nan_embeddings(embeddings_test_flat)

print(embeddings_train_flat.shape)

print(embeddings_test_flat.shape)

(197198, 1536)
(22760, 1536)


In [103]:
def impute_with_flags(df: pd.DataFrame, numeric_cols: list) -> pd.DataFrame:
    """
    Оставляет только числовые колонки.
    Заменяет NaN в числовых колонках нулями и добавляет missing-флаги
    только если в колонке действительно есть пропуски.
    """
    df = df.copy()
    out = pd.DataFrame(index=df.index)  # создаём новый df только с нужными колонками

    for col in numeric_cols:
        out[col] = df[col].fillna(0)  # заменяем NaN на 0
        if df[col].isna().any():      # добавляем флаг только если были NaN
            out[f"{col}_missing"] = df[col].isna().astype(int)
    
    return out

# Пример использования
numeric_columns_test = df_test.select_dtypes(include=[np.number]).columns.tolist()
numeric_columns_train = df_train.select_dtypes(include=[np.number]).columns.tolist()

df_processed_test = impute_with_flags(df_test, numeric_columns_test)
df_processed_train = impute_with_flags(df_train, numeric_columns_train)

print(df_processed_train.head())

       id  resolution  rating_1_count  rating_1_count_missing  rating_2_count  \
0  159385           0             6.0                       0             4.0   
1  288616           0             0.0                       1             0.0   
2  108090           0             0.0                       0             0.0   
3  415607           0             0.0                       1             0.0   
4  332391           0             1.0                       0             0.0   

   rating_2_count_missing  rating_3_count  rating_3_count_missing  \
0                       0             4.0                       0   
1                       1             0.0                       1   
2                       0             1.0                       0   
3                       1             0.0                       1   
4                       0             0.0                       0   

   rating_4_count  rating_4_count_missing  ...  \
0             3.0                       0  ...  

In [73]:
# ====== 1. Dataset ======
class TabularTextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y.values, dtype=torch.float32)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# ====== 2. Model ======
class MLPClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),

            nn.Linear(128, 1)
        )
    def forward(self, x):
        return self.model(x).squeeze(1)

# ====== 3. Training loop ======
def train_model(model, train_loader, val_loader, epochs=20, lr=1e-2, device="cuda", pos_weight=None):
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = optim.AdamW(model.parameters(), lr=lr)

    best_f1 = 0
    best_threshold = 0.5
    best_state = None

    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

        # validation
        model.eval()
        all_preds, all_targets = [], []
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val, y_val = X_val.to(device), y_val.to(device)
                outputs = model(X_val)
                preds = torch.sigmoid(outputs).cpu().numpy()
                all_preds.extend(preds)
                all_targets.extend(y_val.cpu().numpy())

        all_preds = np.array(all_preds)
        all_targets = np.array(all_targets)

        # ищем оптимальный threshold
        thresholds = np.linspace(0.1, 0.9, 17)
        f1_scores = [f1_score(all_targets, (all_preds > t).astype(int)) for t in thresholds]
        max_idx = np.argmax(f1_scores)
        epoch_best_f1 = f1_scores[max_idx]
        epoch_best_threshold = thresholds[max_idx]

        print(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.4f} | Best F1: {epoch_best_f1:.4f} (thr={epoch_best_threshold:.2f})")

        if epoch_best_f1 > best_f1:
            best_f1 = epoch_best_f1
            best_threshold = epoch_best_threshold
            best_state = model.state_dict()

    model.load_state_dict(best_state)
    return model, best_f1, best_threshold

# ====== 4. StratifiedKFold CV ======
def run_cv(features, labels, n_splits=5, batch_size=256, device="cuda"):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    f1_scores = []

    # считаем веса классов для BCEWithLogitsLoss
    best_model = None
    best_f1_global = 0

    for fold, (train_idx, val_idx) in enumerate(skf.split(features, labels)):
        print(f"\n===== Fold {fold+1} =====")

        X_train, X_val = features[train_idx], features[val_idx]
        y_train, y_val = labels.iloc[train_idx], labels.iloc[val_idx]

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

        num_pos = np.sum(y_train == 1)
        num_neg = np.sum(y_train == 0)
        pos_weight = torch.tensor([num_neg / num_pos], dtype=torch.float32).to(device)

        train_ds = TabularTextDataset(X_train, y_train)
        val_ds = TabularTextDataset(X_val, y_val)

        train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

        model = MLPClassifier(input_dim=features.shape[1]).to(device)
        model, best_f1, best_thr = train_model(model, train_loader, val_loader, device=device, pos_weight=pos_weight)

        f1_scores.append(best_f1)
        print(f"Fold {fold+1} | F1: {best_f1:.4f}, thr={best_thr:.2f}")

        if best_f1 > best_f1_global:
            best_model = model
            best_f1_global = best_f1

    print(f"\nMean F1 across folds: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
    return best_model

In [74]:
# ====== 5. Запуск ======
# labels = df_processed_train["resolution"]
# groups = df_train["CommercialTypeName4"]

# tabular_features = df_processed_train.drop(["id","resolution"],axis=1).to_numpy()

# scaler = StandardScaler()
# features = np.concatenate([embeddings_train_flat, tabular_features], axis=1)
# features = scaler.fit_transform(tabular_features)

# run_cv(features, labels, groups, n_splits=5, batch_size=256, device="cuda" if torch.cuda.is_available() else "cpu")

# ====== 5. Запуск ======
labels = df_processed_train["resolution"]

tabular_features = df_processed_train.drop(["id","resolution"],axis=1).to_numpy()
features = np.concatenate([embeddings_train_flat, tabular_features], axis=1)

model = run_cv(features, labels, n_splits=5, batch_size=256, device="cuda" if torch.cuda.is_available() else "cpu")


===== Fold 1 =====
Epoch 1/20 | Loss: 0.3302 | Best F1: 0.6938 (thr=0.90)
Epoch 2/20 | Loss: 0.5170 | Best F1: 0.7344 (thr=0.90)
Epoch 3/20 | Loss: 0.1577 | Best F1: 0.7575 (thr=0.90)
Epoch 4/20 | Loss: 0.1767 | Best F1: 0.7511 (thr=0.90)
Epoch 5/20 | Loss: 0.1513 | Best F1: 0.7596 (thr=0.90)
Epoch 6/20 | Loss: 0.5805 | Best F1: 0.7691 (thr=0.85)
Epoch 7/20 | Loss: 0.2594 | Best F1: 0.7625 (thr=0.85)
Epoch 8/20 | Loss: 0.3320 | Best F1: 0.7416 (thr=0.90)
Epoch 9/20 | Loss: 0.1455 | Best F1: 0.7672 (thr=0.90)
Epoch 10/20 | Loss: 0.4320 | Best F1: 0.7668 (thr=0.90)
Epoch 11/20 | Loss: 0.0959 | Best F1: 0.7749 (thr=0.90)
Epoch 12/20 | Loss: 0.3397 | Best F1: 0.7601 (thr=0.90)
Epoch 13/20 | Loss: 0.3204 | Best F1: 0.7729 (thr=0.90)
Epoch 14/20 | Loss: 0.1622 | Best F1: 0.7745 (thr=0.90)
Epoch 15/20 | Loss: 0.0888 | Best F1: 0.7672 (thr=0.85)
Epoch 16/20 | Loss: 0.2745 | Best F1: 0.7828 (thr=0.90)
Epoch 17/20 | Loss: 0.2920 | Best F1: 0.7791 (thr=0.90)
Epoch 18/20 | Loss: 0.2078 | Best F1:

In [114]:
def run_cv_catboost(features, labels, n_splits=5, random_state=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    f1_scores = []
    best_model = None
    best_f1_global = 0
    best_thr_global = 0.5

    for fold, (train_idx, val_idx) in enumerate(skf.split(features, labels)):
        print(f"\n===== Fold {fold+1} =====")

        X_train, X_val = features[train_idx], features[val_idx]
        y_train, y_val = labels.iloc[train_idx], labels.iloc[val_idx]

        # нормализация внутри фолда
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

        # считаем scale_pos_weight по train
        num_pos = np.sum(y_train == 1)
        num_neg = np.sum(y_train == 0)
        scale_pos_weight = num_neg / num_pos
        print(scale_pos_weight)

        # создаём CatBoostClassifier
        model = CatBoostClassifier(
            iterations=300,
            learning_rate=0.05,
            depth=3,
            loss_function='Logloss',
            eval_metric='F1',
            random_seed=random_state,
            verbose=200,
            use_best_model=True,
            scale_pos_weight=scale_pos_weight   # ✅ УЧЁТ ДИСБАЛАНСА
        )

        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            early_stopping_rounds=100,
            verbose=False
        )

        # получаем вероятности
        val_preds = model.predict_proba(X_val)[:, 1]

        # подбираем threshold
        thresholds = np.linspace(0.1, 0.9, 17)
        f1_scores_thr = [f1_score(y_val, (val_preds > t).astype(int)) for t in thresholds]
        max_idx = np.argmax(f1_scores_thr)
        fold_best_f1 = f1_scores_thr[max_idx]
        fold_best_thr = thresholds[max_idx]

        print(f"Fold {fold+1} | F1: {fold_best_f1:.4f}, thr={fold_best_thr:.2f}")

        f1_scores.append(fold_best_f1)

        if fold_best_f1 > best_f1_global:
            best_model = model
            best_f1_global = fold_best_f1
            best_thr_global = fold_best_thr

    print(f"\nMean F1 across folds: {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
    print(f"Best global F1: {best_f1_global:.4f} @ thr={best_thr_global:.2f}")

    return best_model, best_thr_global

In [115]:
# ====== 5. Запуск ======
labels = df_processed_train["resolution"]

tabular_features = df_processed_train.drop(["id","resolution"],axis=1).to_numpy()
features = np.concatenate([embeddings_train_flat, tabular_features], axis=1)

best_model, best_thr = run_cv_catboost(features, labels, n_splits=5)


===== Fold 1 =====
14.108025282512928
Fold 1 | F1: 0.6956, thr=0.80

===== Fold 2 =====
14.109472272770807
Fold 2 | F1: 0.7038, thr=0.75

===== Fold 3 =====
14.109472272770807
Fold 3 | F1: 0.7003, thr=0.80

===== Fold 4 =====
14.108121049607355
Fold 4 | F1: 0.7006, thr=0.80

===== Fold 5 =====
14.108121049607355
Fold 5 | F1: 0.7064, thr=0.80

Mean F1 across folds: 0.7013 ± 0.0036
Best global F1: 0.7064 @ thr=0.80


In [108]:
def predict_all(model, X_test, test_ids, device="cuda"):
    """
    Делает предсказания для всех объектов теста и формирует DataFrame с колонками id, prediction.
    """
    model.eval()
    model.to(device)

    X_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

    preds = []
    with torch.no_grad():
        for i in range(0, len(X_tensor), 512):  # батчами, чтобы не забить память
            batch = X_tensor[i:i+512]
            prob = torch.sigmoid(model(batch)).squeeze().cpu().numpy()
            preds.append(prob)

    preds = np.concatenate(preds)

    # бинаризация по порогу 0.5
    preds_bin = (preds >= 0.5).astype(int)

    # собираем финальную таблицу
    submission = pd.DataFrame({
        "id": test_ids,
        "prediction": preds_bin
    })

    submission.to_csv("submission.csv", index=False)
    print("✅ submission.csv сохранён!")
    return submission

In [110]:
# ====== 5. Запуск ======

test_ids = df_processed_test["id"]
tabular_features = df_processed_test.drop(["id","PriceDiscounted_missing"],axis=1).to_numpy()
X_test = np.concatenate([embeddings_test_flat, tabular_features], axis=1)

submission = predict_all(model, X_test, test_ids, device="cuda")

✅ submission.csv сохранён!


In [116]:
def predict_all_catboost(model, X_test, test_ids, threshold=0.5, scaler=None):
    """
    Делает предсказания для CatBoostClassifier и формирует DataFrame с колонками id, prediction.
    threshold — оптимальный порог, найденный на валидации.
    scaler — StandardScaler, если применялся при обучении.
    """

    # нормализация, если была
    if scaler is not None:
        X_test = scaler.transform(X_test)

    # вероятности для класса 1
    preds = model.predict_proba(X_test)[:, 1]

    # бинаризация по threshold
    preds_bin = (preds > threshold).astype(int)

    # собираем финальную таблицу
    submission = pd.DataFrame({
        "id": test_ids,
        "prediction": preds_bin
    })

    submission.to_csv("submission.csv", index=False)
    print("✅ submission.csv сохранён!")
    return submission


In [117]:
# 2. собираем тестовые признаки
test_ids = df_processed_test["id"]
tabular_features = df_processed_test.drop(["id","PriceDiscounted_missing"], axis=1).to_numpy()
X_test = np.concatenate([embeddings_test_flat, tabular_features], axis=1)

# 3. предсказания
submission = predict_all_catboost(best_model, X_test, test_ids, threshold=best_thr)

✅ submission.csv сохранён!


In [18]:
labels.value_counts(normalize=True)
df_train.groupby("CommercialTypeName4")["resolution"].mean()


CommercialTypeName4
3D очки для телевизора                     0.000000
3D принтер (не ручка)                      0.000000
Audio/video кассеты                        0.040000
Bluetooth адаптер                          0.000000
Bluetooth метка                            0.015152
                                             ...   
Электронный модуль                         0.026042
Электронный переводчик                     0.000000
Электрооткрывалка, штопор электрический    0.461538
Электросковорода                           0.333333
Этнический музыкальный инструмент          0.000000
Name: resolution, Length: 634, dtype: float64

In [43]:
# print("features shape:", features.shape)
# print("labels unique:", labels.unique() if hasattr(labels, "unique") else np.unique(labels))

print("NaN in embeddings_train_flat:", np.isnan(embeddings_train_flat).any())
# print("Inf in features:", np.isinf(tabular_features).any())
# print("NaN in labels:", labels.isna().any() if hasattr(labels, "isna") else np.isnan(labels).any())

def find_nan_embeddings(embeddings: np.ndarray):
    """
    Находит NaN внутри эмбеддингов.
    
    Args:
        embeddings: np.ndarray формы (N, D)

    Returns:
        список индексов с NaN и сами строки
    """
    nan_mask = np.isnan(embeddings).any(axis=1)  # True, если в строке есть NaN
    nan_indices = np.where(nan_mask)[0]          # индексы таких строк
    
    # for idx in nan_indices:
        # print(f"⚠️ Embedding {idx} содержит NaN:\n{embeddings[idx]}\n")
    
    return nan_indices

# пример
nan_indices = find_nan_embeddings(embeddings_train_flat)
print("Всего строк с NaN:", len(nan_indices))




NaN in embeddings_train_flat: False
Всего строк с NaN: 0


In [6]:
import pandas as pd
import os
import glob
import chardet

def detect_encoding(file_path, sample_size=100000):
    """Определяет кодировку файла"""
    with open(file_path, "rb") as f:
        raw = f.read(sample_size)
    result = chardet.detect(raw)
    return result["encoding"]

def analyze_csv(file_path):
    """Читает CSV и выводит ML-ориентированный отчёт"""
    print("="*80)
    print(f"Файл: {file_path}")

    # Определение кодировки
    encoding = detect_encoding(file_path)
    print(f"Определённая кодировка: {encoding}")

    try:
        df = pd.read_csv(file_path, encoding=encoding)
    except Exception as e:
        print(f"Ошибка чтения {file_path}: {e}")
        return None

    # Размер
    print(f"Размер: {df.shape[0]} строк, {df.shape[1]} столбцов\n")

    # Информация о столбцах
    info_lines = []
    for col in df.columns:
        col_type = df[col].dtype
        missing = df[col].isna().sum()
        unique_vals = df[col].nunique()
        info_lines.append(f"- {col}: {col_type}, пропусков: {missing}, уникальных: {unique_vals}")

    print("Столбцы:")
    print("\n".join(info_lines))

    # Базовая статистика
    print("\nСтатистика (числовые признаки):")
    print(df.describe().T)

    print("\nСтатистика (категориальные признаки):")
    cat_cols = df.select_dtypes(include=["object", "category"]).columns
    for col in cat_cols:
        top_vals = df[col].value_counts().head(5)
        print(f"\n{col} (топ-5 значений):")
        print(top_vals)

    return df

if __name__ == "__main__":
    # Берём все CSV в текущей папке
    csv_files = glob.glob("*.csv")

    if not csv_files:
        print("В папке нет CSV файлов.")
    else:
        for file in csv_files:
            analyze_csv(file)


Файл: ml_ozon_counterfeit_test_utf8.csv
Определённая кодировка: utf-8
Размер: 22760 строк, 44 столбцов

Столбцы:
- id: int64, пропусков: 0, уникальных: 22760
- brand_name: object, пропусков: 5832, уникальных: 863
- description: object, пропусков: 2224, уникальных: 13512
- name_rus: object, пропусков: 0, уникальных: 18441
- CommercialTypeName4: object, пропусков: 0, уникальных: 440
- rating_1_count: float64, пропусков: 21053, уникальных: 92
- rating_2_count: float64, пропусков: 21053, уникальных: 47
- rating_3_count: float64, пропусков: 21053, уникальных: 74
- rating_4_count: float64, пропусков: 21053, уникальных: 93
- rating_5_count: float64, пропусков: 21053, уникальных: 303
- comments_published_count: float64, пропусков: 21053, уникальных: 262
- photos_published_count: float64, пропусков: 21053, уникальных: 209
- videos_published_count: float64, пропусков: 21053, уникальных: 79
- PriceDiscounted: float64, пропусков: 3, уникальных: 22757
- item_time_alive: int64, пропусков: 0, уникаль